In [ ]:
# *******************************************************
#          <<<   Python ERDDAP Interface  >>>           #
#                                                       #
#      Functions:                                       #
#       (a) search by catagory, protocol, keyword       #
#       (b) request / process data from search / url    #
#       (c) examine/QC metadata and other information   #
#       (d) create, modify ScienceBase record           #
#                                                       #
#       Simple prototyping and tasks, for the moment    #
#                                                       #
#       T. Wellman, BCB, Core Sciences, USGS            #
#       code snippets for future ideas                  #
# *******************************************************

# DRAFT NOTES (DIARY):

# Below are code snippets related to processing, future ideas, and refinements
# This is a work in progress........ 




In [ ]:
## 

***************************************************************************************************************
******   " The section below has ScienceBase code snippets to create, modify, retrieve information "     ******
***************************************************************************************************************




In [30]:
# login to ScienceBase using pysb. retrieve item
import pysb

login_name = 'twellman@usgs.gov'

sb = pysb.SbSession()
sb.loginc(str(login_name))


········


<pysb.SbSession.SbSession instance at 0x104573290>

In [88]:
# Delete child items

import pprint

print('my id', sb.get_my_items_id())

item_ids = sb.get_child_ids(sb.get_my_items_id())
sb.delete_items(item_ids) 
print 'done deleting child items'


('my id', u'570c0592e4b0ef3b7ca04e9e')
done deleting child items


In [29]:
test = sb.get_item('5898f72ce4b0efcedb707861')
print test['contacts']
print test[u'tags']
# g = [y for y in (tuple(x.items()) for x in test['contacts'])]
seen = set()
g = [item for item in test['contacts'] if item['name'] not in seen and not seen.add(item['name'])]
print seen

[{u'name': u'Bob Simons', u'contactType': u'person', u'primaryLocation': {u'streetAddress': {u'city': u'Monterey', u'state': u'CA', u'zip': u'93940', u'line1': u'99 Pacific St, Suite 255A', u'country': u'USA'}, u'officePhone': u'+1 831-658-3205'}, u'organization': {u'displayText': u'NOAA NMFS SWFSC Environmental Research Division'}, u'type': u'Point of Contact', u'email': u'bob.simons@noaa.gov'}, {u'name': u'NOAA NMFS SWFSC ERD', u'onlineResource': u'http://www.nps.gov/chis/naturescience/index.htm', u'contactType': u'person', u'primaryLocation': {u'streetAddress': {}}, u'organization': {u'displayText': u'CINP'}, u'type': u'Point of Contact', u'email': u'Roy.Mendelssohn@noaa.gov'}, {u'oldPartyId': 23179, u'name': u'Abigail L Benson', u'firstName': u'Abigail', u'middleName': u'L', u'jobTitle': u'Biologist', u'lastName': u'Benson', u'primaryLocation': {u'building': u'DFC Bldg 810', u'faxPhone': u'3032024229', u'name': u'Abigial L Benson/GIO/USGS/DOI - Primary Location', u'streetAddress': 

In [37]:
# new_item_info = {'title': "title",
#                    'parentId': sb.get_my_items_id(),
#                    'provenance': {'annotation': 'test'}}
# test = sb.create_item(new_item_info)
SB_rec = sb.get_item('58935856e4b0fa1e59b730f9')
#sb.updateSbItem(SB_rec)
#sb.update_item(SB_rec)
for f in SB_rec['files']:
    print f['name']


PMNM_FISH_BLT_712b_5843_9069.csv
PMNM_FISH_BLT_iso19115.xml


In [34]:
import pprint
# find_items_by_any_text(text) Find items containing the given text somewhere in the item.
# find_items_by_title(text) Find items containing the given text in the title of the item.
# find_items(params) Find items meeting the criteria in the specified search parameters. 
# These are the same parameters that you pass to ScienceBase in an advanced search.

# results = sb.find_items_by_any_text('1995 Florida Keys Reef Visual Census')

#bulk_search = sb.find_items_by_any_text("PMNM_FISH_BLT_712b_5843_9069.csv") # 'PMNM_FISH_BLT'
bulk_search = sb.find_items_by_title('1995 Florida Keys Reef Visual Census') # 'PMNM_FISH_BLT'


search_id = []
for item in bulk_search[u'items']:
        search_id.append(item['id'])
    

pprint.pprint(search_id)  

[]


In [57]:
# bulk_search = sb.find_items_by_title(SB_rec["title"])
SB_rec = sb.get_item('588d7e55e4b08c8121c909e3')
bulk_search = sb.find_items_by_any_text(SB_rec["title"])
for item in bulk_search[u'items']:
    print item['id']



588d6480e4b08c8121c909cd
588d633ee4b08c8121c909ca
588d5997e4b08c8121c909c7
57fe9d79e4b0824b2d14f219
5882ef94e4b0d0023163767c
588d7e55e4b08c8121c909e3
588d9789e4b08c8121c91a61
588d7bb5e4b08c8121c909db
588d7de3e4b08c8121c909df


In [45]:
import sys
SB_rec = sb.get_item('5898f72ce4b0efcedb707861')

# check for potential duplicate items in sciencebase
sys.stdout.write("\n\t{}{}".format('Searching for duplicate', 
        ' sb items using filenames and item title'))  
search_id = []

bulk_search = sb.find_items_by_title(SB_rec["title"])

for item in bulk_search[u'items']:
    if item['id'] not in search_id:
        search_id.append(item['id'])
if 'files' in SB_rec:
    for f in SB_rec['files']:
        blk_search = sb.find_items_by_any_text(f['name'])
        for item in bulk_search[u'items']:
            if item['id'] not in search_id:
                search_id.append(item['id'])
if search_id:
    sys.stdout.write("\n\t\t{}".format('*** WARNING potential duplicate items ***'))
    for s in search_id:
            sys.stdout.write("\n\t\t\t{}".format(s))
else:
    sys.stdout.write("\n\t\t{}".format('pass: item duplicates not dentified'))
    
#i = d.iteritems()   
type(bulk_search[u'items'][0])


	Searching for duplicate sb items using filenames and item title
		*** WARNING potential duplicate items ***
			5898f72ce4b0efcedb707861

dict

In [57]:
dataset = '1994fk'
data = {}
data["sb_id"] = '5898f72ce4b0efcedb707861'

print 'current id', SB_rec['id']

# [item in bulk_search[u'items'].iteritems]
blk_search = sb.find_items_by_title(SB_rec["title"])[u'items']
if 'files' in SB_rec:
    for f in SB_rec['files']:
        blk_search.extend(sb.find_items_by_any_text(f['name'])[u'items'])        
    blk_search.extend(sb.find_items_by_any_text(dataset)[u'items'])  
    seen = set()
    search_id = [item['id'] for item in blk_search 
                if item['id'] not in seen and not seen.add(item['id'])]
    search_id = filter(lambda a: a != data["sb_id"], search_id )
search_id

current id 5898f72ce4b0efcedb707861


[]

In [21]:
import json

tempdir = '/Users/twellman/Documents/BCB_data_projects/OBIS_usa_database/erddap_MBON_test/'
sb_json = "file_sb.json"

with open(tempdir + sb_json) as fp:    
        SB_status = json.load(fp)
    
for d in SB_status[2]["datasets (id)"]:
    if SB_status[2]["datasets (id)"][d]["sb_id"] != "None":
        print "ScienceBase ID: ", SB_status[2]["datasets (id)"][d]["sb_id"]
        sbitem = sb.get_item(SB_status[2]["datasets (id)"][d]["sb_id"])
        contact_cpy = []
        for i in sbitem['contacts']:
            if i['name'] != 'Marine Biodiversity Oservation Network':
                contact_cpy.append(i)
        sbitem['contacts'] = contact_cpy        
        sb.updateSbItem(sbitem)  

ScienceBase ID:  57fe7ca1e4b0824b2d146914
ScienceBase ID:  57fe9025e4b0824b2d14bd7c
ScienceBase ID:  57fe93d5e4b0824b2d14cbe1
ScienceBase ID:  57fe9410e4b0824b2d14cce5
ScienceBase ID:  57fe9444e4b0824b2d14cdbb
ScienceBase ID:  57fe986ae4b0824b2d14de59
ScienceBase ID:  57fe9872e4b0824b2d14de61
ScienceBase ID:  587f8830e4b085de6c11f242
ScienceBase ID:  57fe9883e4b0824b2d14ded5
ScienceBase ID:  57fe9810e4b0824b2d14dd24
ScienceBase ID:  57fe988be4b0824b2d14dedd
ScienceBase ID:  57fe98e8e4b0824b2d14e072
ScienceBase ID:  57fe9980e4b0824b2d14e272
ScienceBase ID:  57fe99a9e4b0824b2d14e2f2
ScienceBase ID:  57fe99f7e4b0824b2d14e3f3
ScienceBase ID:  57fe9a50e4b0824b2d14e542
ScienceBase ID:  57fe9a9de4b0824b2d14e650
ScienceBase ID:  57fe9af0e4b0824b2d14e7a7
ScienceBase ID:  57fe9af8e4b0824b2d14e7af
ScienceBase ID:  57fe9affe4b0824b2d14e7fe
ScienceBase ID:  57fe9b07e4b0824b2d14e823
ScienceBase ID:  57fe9b0ee4b0824b2d14e82b
ScienceBase ID:  57fe9b16e4b0824b2d14e835
ScienceBase ID:  57fe9b20e4b0824b2

In [ ]:
# simple print out a list of completed file downloads
# from sb_json file. At present tweak adhoc for print out
# includes hyperlinks (can cut and paste into doc)



with open(tempdir + sb_json) as fp:    
        SB_status = json.load(fp)
    
dataset = 0
for d in SB_status[2]["datasets (id)"]:
    if SB_status[2]["datasets (id)"][d]["sb_id"] != "None":
        dataset = dataset  + 1
        print("Dataset number: ", dataset)
        repl = SB_status[2]["datasets (id)"][d]['dataset_url'] 
        SB_status[2]['datasets (id)'][d]['dataset_url'] = repl
        print "Dataset name:", d
        print "ScienceBase ID: ", SB_status[2]["datasets (id)"][d]["sb_id"]
        print "ScienceBase url:", "https://www.sciencebase.gov/catalog/item/" + SB_status[2]["datasets (id)"][d]["sb_id"]
        print "dataset_url: ", repl
        print

In [11]:
# move sb items to parent ID
# move_item(itemid, parentid) Move the ScienceBase Item with the given itemid under the ScienceBase Item with the given parentid.
# move_items(itemids, parentid) Move all of the ScienceBase Items with the given itemids under the ScienceBase Item with the given parentid.


import json

tempdir = '/Users/twellman/Documents/BCB_data_projects/OBIS_usa_database/erddap_MBON_test/'
sb_json = "file_sb.json"
with open(tempdir + sb_json) as fp:    
        SB_status = json.load(fp)
    
dataset = 0
itemsb = []
for d in SB_status[2]["datasets (id)"]:
    if SB_status[2]["datasets (id)"][d]["sb_id"] != "None":
        dataset = dataset  + 1
        print("Dataset number: ", dataset)
        repl = SB_status[2]["datasets (id)"][d]['dataset_url'] 
        SB_status[2]['datasets (id)'][d]['dataset_url'] = repl
        print "Dataset name:", d
        print "ScienceBase ID: ", SB_status[2]["datasets (id)"][d]["sb_id"]
        print "ScienceBase url:", "https://www.sciencebase.gov/catalog/item/" + SB_status[2]["datasets (id)"][d]["sb_id"]
        print "dataset_url: ", repl
        print
        itemsb.append(SB_status[2]["datasets (id)"][d]["sb_id"])
        
# move items to OBIS
# sb.move_items(itemsb, '579b64c6e4b0589fa1c98118') 

('Dataset number: ', 1)
Dataset name: dt2008
ScienceBase ID:  57fe7ca1e4b0824b2d146914
ScienceBase url: https://www.sciencebase.gov/catalog/item/57fe7ca1e4b0824b2d146914
dataset_url:  http://gcoos4.tamu.edu:8080/erddap/tabledap/dt2008.html

('Dataset number: ', 2)
Dataset name: dt2000
ScienceBase ID:  57fe9025e4b0824b2d14bd7c
ScienceBase url: https://www.sciencebase.gov/catalog/item/57fe9025e4b0824b2d14bd7c
dataset_url:  http://gcoos4.tamu.edu:8080/erddap/tabledap/dt2000.html

('Dataset number: ', 3)
Dataset name: dt2004
ScienceBase ID:  57fe93d5e4b0824b2d14cbe1
ScienceBase url: https://www.sciencebase.gov/catalog/item/57fe93d5e4b0824b2d14cbe1
dataset_url:  http://gcoos4.tamu.edu:8080/erddap/tabledap/dt2004.html

('Dataset number: ', 4)
Dataset name: dt2006
ScienceBase ID:  57fe9410e4b0824b2d14cce5
ScienceBase url: https://www.sciencebase.gov/catalog/item/57fe9410e4b0824b2d14cce5
dataset_url:  http://gcoos4.tamu.edu:8080/erddap/tabledap/dt2006.html

('Dataset number: ', 5)
Dataset name

[{u'body': u"In 1998, the Florida Fish and Wildlife Conservation Commission's (FWC) Fisheries Independent Monitoring (FIM) program began a long-term monitoring effort of key reef fish populations in the Florida Keys National Marine Sanctuary. This effort was aimed at evaluating the relative abundance, size structure, and habitat utilization of specific reef fish species that are targeted by commercial and recreational fisheries.; Smith, S.G., et al. 2011, Multispecies survey design for assessing reef-fish stocks, spatially explicit management performance, and ecosystem condition. Fisheries Research 109(2011)25-41; Brandt, M.E., et. al. 2009, A Cooperative Multi-agency Reef Fish Monitoring Protocol for the Florida Keys Coral Reef Ecosystem. Retrieve from http://www.coris.noaa.gov/activities/fish_monitoring_protocol/&nbsp; &nbsp;\n<br> \n<br>Unrestricted use and distribution",
  u'browseCategories': [u'Data'],
  u'browseTypes': [u'NetCDF OPeNDAP Service'],
  u'citation': u'Data citation 

In [83]:
import pandas as pd
from IPython.core.display import display
import requests

pd.set_option('display.max_rows', 500)

url_b = 'http://gcoos4.tamu.edu:8080/erddap/info/index.json'
info_req = requests.get(url_b).json()
info_file = pd.DataFrame(info_req['table']['rows'], columns=info_req['table']['columnNames'])     
display(info_file) 

,griddap,Subset,tabledap,Make A Graph,wms,files,Title,Summary,FGDC,ISO 19115,Info,Background Info,RSS,Email,Institution,Dataset ID
0,,http://gcoos4.tamu.edu:8080/erddap/tabledap/al...,http://gcoos4.tamu.edu:8080/erddap/tabledap/al...,http://gcoos4.tamu.edu:8080/erddap/tabledap/al...,,,* The List of All Active Datasets in this ERDD...,This dataset is a table which has a row of inf...,,,http://gcoos4.tamu.edu:8080/erddap/info/allDat...,http://gcoos4.tamu.edu:8080/erddap,,,TAMU Oceanography,allDatasets
1,,http://gcoos4.tamu.edu:8080/erddap/tabledap/fk...,http://gcoos4.tamu.edu:8080/erddap/tabledap/fk...,http://gcoos4.tamu.edu:8080/erddap/tabledap/fk...,,,"1994 Florida Keys Reef Visual Census, v3.3","In 1998, the Florida Fish and Wildlife Conserv...",http://gcoos4.tamu.edu:8080/erddap/metadata/fg...,http://gcoos4.tamu.edu:8080/erddap/metadata/is...,http://gcoos4.tamu.edu:8080/erddap/info/fk1994...,???,http://gcoos4.tamu.edu:8080/erddap/rss/fk1994.rss,http://gcoos4.tamu.edu:8080/erddap/subscriptio...,Marine Biodiversity Observation Network,fk1994
2,,http://gcoos4.tamu.edu:8080/erddap/tabledap/fk...,http://gcoos4.tamu.edu:8080/erddap/tabledap/fk...,http://gcoos4.tamu.edu:8080/erddap/tabledap/fk...,,,"1995 Florida Keys Reef Visual Census, v3.3","In 1998, the Florida Fish and Wildlife Conserv...",http://gcoos4.tamu.edu:8080/erddap/metadata/fg...,http://gcoos4.tamu.edu:8080/erddap/metadata/is...,http://gcoos4.tamu.edu:8080/erddap/info/fk1995...,???,http://gcoos4.tamu.edu:8080/erddap/rss/fk1995.rss,http://gcoos4.tamu.edu:8080/erddap/subscriptio...,Marine Biodiversity Observation Network,fk1995
3,,http://gcoos4.tamu.edu:8080/erddap/tabledap/cr...,http://gcoos4.tamu.edu:8080/erddap/tabledap/cr...,http://gcoos4.tamu.edu:8080/erddap/tabledap/cr...,,,1996 Florida Keys Coral Reef Monitoring Project,The purpose of the Coral Reef Evaluation and M...,http://gcoos4.tamu.edu:8080/erddap/metadata/fg...,http://gcoos4.tamu.edu:8080/erddap/metadata/is...,http://gcoos4.tamu.edu:8080/erddap/info/cremp_...,???,http://gcoos4.tamu.edu:8080/erddap/rss/cremp_1...,http://gcoos4.tamu.edu:8080/erddap/subscriptio...,FWRI,cremp_1996
4,,http://gcoos4.tamu.edu:8080/erddap/tabledap/fk...,http://gcoos4.tamu.edu:8080/erddap/tabledap/fk...,http://gcoos4.tamu.edu:8080/erddap/tabledap/fk...,,,"1996 Florida Keys Reef Visual Census, v3.3","In 1998, the Florida Fish and Wildlife Conserv...",http://gcoos4.tamu.edu:8080/erddap/metadata/fg...,http://gcoos4.tamu.edu:8080/erddap/metadata/is...,http://gcoos4.tamu.edu:8080/erddap/info/fk1996...,???,http://gcoos4.tamu.edu:8080/erddap/rss/fk1996.rss,http://gcoos4.tamu.edu:8080/erddap/subscriptio...,Marine Biodiversity Observation Network,fk1996
5,,http://gcoos4.tamu.edu:8080/erddap/tabledap/cr...,http://gcoos4.tamu.edu:8080/erddap/tabledap/cr...,http://gcoos4.tamu.edu:8080/erddap/tabledap/cr...,,,1997 Florida Keys Coral Reef Monitoring Project,The purpose of the Coral Reef Evaluation and M...,http://gcoos4.tamu.edu:8080/erddap/metadata/fg...,http://gcoos4.tamu.edu:8080/erddap/metadata/is...,http://gcoos4.tamu.edu:8080/erddap/info/cremp_...,???,http://gcoos4.tamu.edu:8080/erddap/rss/cremp_1...,http://gcoos4.tamu.edu:8080/erddap/subscriptio...,FWRI,cremp_1997
6,,http://gcoos4.tamu.edu:8080/erddap/tabledap/fk...,http://gcoos4.tamu.edu:8080/erddap/tabledap/fk...,http://gcoos4.tamu.edu:8080/erddap/tabledap/fk...,,,"1997 Florida Keys Reef Visual Census, v3.3","In 1998, the Florida Fish and Wildlife Conserv...",http://gcoos4.tamu.edu:8080/erddap/metadata/fg...,http://gcoos4.tamu.edu:8080/erddap/metadata/is...,http://gcoos4.tamu.edu:8080/erddap/info/fk1997...,???,http://gcoos4.tamu.edu:8080/erddap/rss/fk1997.rss,http://gcoos4.tamu.edu:8080/erddap/subscriptio...,Marine Biodiversity Observation Network,fk1997
7,,http://gcoos4.tamu.edu:8080/erddap/tabledap/cr...,http://gcoos4.tamu.edu:8080/erddap/tabledap/cr...,http://gcoos4.tamu.edu:8080/erddap/tabledap/cr...,,,1998 Florida Keys Coral Reef Monitoring Project,The purpose of the Coral Reef Eval

In [ ]:
# view pandas dataframe of table of datasets requested

pd.set_option('display.max_colwidth', -1)
info_frame = pd.DataFrame(info_request['table']['rows'], columns=info_request['table']['columnNames'])
    
display(info_frame) 


In [ ]:
# fiddling with Asynchronous HTTP client

from tornado import ioloop, httpclient

i = 0

def handle_request(response):
    print(response.code)
    global i
    i -= 1
    if i == 0:
        ioloop.IOLoop.instance().stop()

http_client = httpclient.AsyncHTTPClient()
for url in open('urls.txt'):
    i += 1
    http_client.fetch(url.strip(), handle_request, method='HEAD')
ioloop.IOLoop.instance().start()

In [ ]:
# Generate uuid and time generation
import uuid
import timeit
timer=timeit.Timer('uuid.uuid1()','import uuid')
timer.repeat(3, 300000)

In [ ]:
# Curl process 

import subprocess
params = "name="+name+"&tags="+tags+"&description="+description+"&status="+status
proc = subprocess.Popen(['curl', '-X', 'POST', '-d', 
                              params, url], stdout=subprocess.PIPE)
(out, err) = proc.communicate()
print out,err

In [ ]:
%%javascript 
var kernel = Jupyter.notebook.kernel; 
var command = ["notebookPath = ",
               "'", window.document.body.dataset.notebookPath, "'" ].join('')
//alert(command)
kernel.execute(command)
var command = ["notebookName = ",
               "'", window.document.body.dataset.notebookName, "'" ].join('')
//alert(command)
kernel.execute(command)

In [ ]:
import os
p = str(os.getcwd()) 
print p + "/" + notebookName

In [ ]:
# first revised way to read errdap contact info from information file
# later streamlined to one call

            if info_file['Attribute Name'].str.contains("pointOfContact_name").any():
                 store_name.append(proc_erddap_contacts('pointOfContact', info_file,  u'Point of Contact'),
                                   info_dict['info_keys'], info_dict['label_keys'])
            if info_file['Attribute Name'].str.contains("originator_name").any():
                 store_name.append(proc_erddap_contacts('originator', info_file, u'Originator'),
                                   info_dict['info_keys'], info_dict['label_keys'])
            if info_file['Attribute Name'].str.contains("processor_name").any():
                 store_name.append(proc_erddap_contacts('processor', info_file,  u'Processor'),
                                   info_dict['info_keys'], info_dict['label_keys'])
            if info_file['Attribute Name'].str.contains("publisher_name").any():
                 store_name.append(proc_erddap_contacts('publisher', info_file,  u'Publisher'),
                                   info_dict['info_keys'], info_dict['label_keys'])

In [ ]:
# old method of autopopulating and updating contact info in sciencebase
# new method in main code pulls from the ERDDAP info file and ignores metadata file contacts

# OLD METHOD - edit and remove redundancy in contact names, a bit hardwired, work in progress
            #store_name = []
            #if SB_rec['contacts']:
            #    for s in SB_rec['contacts']:
            #        if s['name'] != '' and '?' not in s['name']:
            #            store_name.append(s)
            #if nf_dict['BCB_contact']['name'] not in store_name:
            #    store_name.append(nf_dict['BCB_contact'])
            #if nf_dict['processor']['name'] not in store_name:
            #    store_name.append(nf_dict['processor'])
            #if nf_dict['publisher']['name'] not in store_name:
            #    store_name.append(nf_dict['publisher'] )
            #seen = set()            
            #new_store_name = []
            #for s in store_name:
            #    if s['name']  not in seen:
            #        seen.add(s['name'])
            #        new_store_name.append(s) 
            #SB_rec['contacts'] = new_store_name


In [ ]:
# *****************************************************

#   code below started, large work in progress
                         
#   Generate input/log file of manual search, for re-runs and reproduction - NOT yet implemented **
#   Add Header username, computer info, datestamp add logging capabilities
#   class Logger(object):
#      def __init__(self, filename="Default.log"):
#          self.terminal = sys.stdout
#          self.log = open(filename, "a")
#
#      def write(self, message):
#          self.terminal.write(message)
#          self.log.write(message)
#   sys.stdout = Logger("yourlogfilename.txt")

# Request upper-level info on available datasets incomplete **
# def reqtable(df, name):
#    global df_result
#    url = ''.join(df.loc[df['Resource'] == name]['URL'])
#    response, err = url_request(url,'json')
#    df_result = pd.DataFrame(response['table']['rows'], columns=response['table']['columnNames']) 
#    df_result.drop(df_result[df_result['Dataset ID'] == "allDatasets"].index)
#    get_stats(df_result)      
#    return
    
# Evaluate table of requested datasets, flag or exclude datasets as warranted.  incomplete - not started **
# def evaltable(df_result):
#    print("... incomplete, may perform assessments in the future")
#    return

# Evaluate individual datasets, flag or exclude as warranted, incomplete - not started **
# def evaldataset(df_result):
#    print("Functionality may include QA/QC metrics, taxonomic checks, and other criteria")
#    return

In [ ]:
# reset sbjson status file
def reset_sbjson(tempdir, baseurl):
    status = [{'general information': "none"},{'local_directory': tempdir},{'datasets (id)': {}}] 
    status[0]['general information'] = nf_dict[u'general information']
    status[0]['general information'][3]['data base url'] = baseurl 
    date = datetime.datetime.strftime(datetime.datetime.now(), '%Y-%m-%d %H:%M:%S')
    status[0]['general information'][1]['file created (date-time)'] = date
    with open(file_sb, 'w') as fp:
        json.dump(status, fp, indent=4)
    return status

In [31]:
sb_status = [
    {
        "general information": [
            {
                "purpose": "Data Distillery development: MBON (GCOOS) data downloads"
            }, 
            {
                "file created (date-time)": "2017-01-28 20:51:04"
            }, 
            {
                "file modified (date-time)": "2017-01-28 20:51:47"
            }, 
            {
                "data base url": "http://gcoos4.tamu.edu:8080/erddap"
            }, 
            {
                "data search url": "http://gcoos4.tamu.edu:8080/erddap/search/advanced.html?searchFor=obis&protocol=%28any%29&cdm_data_type=%28any%29&institution=%28any%29&ioos_category=biology&keywords=%28any%29&long_name=%28any%29&standard_name=%28any%29&variableName=%28any%29&maxLon=&minLon=&minTime=&minLat=&maxLat=&maxTime="
            }, 
            {
                "data search json": "http://gcoos4.tamu.edu:8080/erddap/search/advanced.json?searchFor=obis&protocol=%28any%29&cdm_data_type=%28any%29&institution=%28any%29&ioos_category=biology&keywords=%28any%29&long_name=%28any%29&standard_name=%28any%29&variableName=%28any%29&maxLon=&minLon=&minTime=&minLat=&maxLat=&maxTime="
            }, 
            {
                "contacts": "Abigail Benson, Tristan P. Wellman, Steve Aulenbach, Sky Bristol"
            }, 
            {
                "email": "albenson@usgs.gov, twellman@usgs.gov, saulenbach@usgs.gov, sbristol@usgs.gov"
            }, 
            {
                "organization": "U.S. Geological Survey, Core Science Analytics, and Synthesis (B.C.B.)"
            }, 
            {
                "address": "Denver Federal Center, Building 810, Lakewood, Colorado"
            }, 
            {
                "processing uuid": 12334
            }
        ]
        }
    ]

In [18]:
"processing uuid" in sb_status[0]["general information"]

False

In [28]:
if len(sb_status[0]["general information"]) == 10:
        sb_status[0]["general information"].append({'processing uuid': uuid.uuid1()})
    elif not SB_status[0][u'general information'][10]['processing uuid']:
        SB_status[0][u'general information'][10]['processing uuid'] = uuid.uuid1()

10 here


In [32]:
if len(sb_status[0]["general information"]) == 10:
    sb_status[0]["general information"].append({'test': 1234})

In [33]:
sb_status[0]["general information"]

[{'purpose': 'Data Distillery development: MBON (GCOOS) data downloads'},
 {'file created (date-time)': '2017-01-28 20:51:04'},
 {'file modified (date-time)': '2017-01-28 20:51:47'},
 {'data base url': 'http://gcoos4.tamu.edu:8080/erddap'},
 {'data search url': 'http://gcoos4.tamu.edu:8080/erddap/search/advanced.html?searchFor=obis&protocol=%28any%29&cdm_data_type=%28any%29&institution=%28any%29&ioos_category=biology&keywords=%28any%29&long_name=%28any%29&standard_name=%28any%29&variableName=%28any%29&maxLon=&minLon=&minTime=&minLat=&maxLat=&maxTime='},
 {'data search json': 'http://gcoos4.tamu.edu:8080/erddap/search/advanced.json?searchFor=obis&protocol=%28any%29&cdm_data_type=%28any%29&institution=%28any%29&ioos_category=biology&keywords=%28any%29&long_name=%28any%29&standard_name=%28any%29&variableName=%28any%29&maxLon=&minLon=&minTime=&minLat=&maxLat=&maxTime='},
 {'contacts': 'Abigail Benson, Tristan P. Wellman, Steve Aulenbach, Sky Bristol'},
 {'email': 'albenson@usgs.gov, twellm

In [34]:
len(sb_status[0]["general information"])

11

In [82]:
x = [1,2,3,2,2,2,3,4]
x = filter(lambda a: a != 2, x)
x

[1, 3, 3, 4]

In [89]:
files = {
                    "info_dict": "/Users/twellman/Documents/BCB_data_projects/OBIS_usa_database/NOAA/aadcArgos/info_dict.json",
                    ".csv": null,
                    ".nc": null,
                    ".ncHeader": null,
                    ".iso19115": "/Users/twellman/Documents/BCB_data_projects/OBIS_usa_database/NOAA/aadcArgos/aadcArgos_iso19115.xml",
                    "info_request": "/Users/twellman/Documents/BCB_data_projects/OBIS_usa_database/NOAA/aadcArgos/info_request.json"
                }

NameError: name 'null' is not defined

In [11]:
5898f72ce4b0efcedb707861

[1, 23, 4, ['r', '66', 66]]

In [ ]:
      # check for potential duplicate items in sciencebase (title, filename, dataset)
            sys.stdout.write("\n\t{}{}".format('Searching for duplicate', 
                ' sb items using filenames, item title, and dataset ID'))  
        #  ..... duplicate search by title
            search_id = sb.find_items_by_title(SB_rec["title"])      
        #  ..... duplicate search by filename(s)
            if 'files' in SB_rec:
                for f in SB_rec['files']:
                    search_id.extend(sb.find_items_by_any_text(f['name']))
        #  ..... duplicate search by dataset id 
            search_id.extend(sb.find_items_by_any_text(dataset))      # check for potential duplicate items in sciencebase (title, filename, dataset)
            sys.stdout.write("\n\t{}{}".format('Searching for duplicate', 
                ' sb items using filenames, item title, and dataset ID'))  
        #  ..... duplicate search by title
            search_id = sb.find_items_by_title(SB_rec["title"])      
        #  ..... duplicate search by filename(s)
            if 'files' in SB_rec:
                for f in SB_rec['files']:
                    search_id.extend(sb.find_items_by_any_text(f['name']))
        #  ..... duplicate search by dataset id 
            search_id.extend(sb.find_items_by_any_text(dataset))